## Example Jupyter Notebook with LLM in LangGraph

In [ ]:
# Install necessary packages if you haven't
# !pip install langgraph openai

import openai
from langgraph import LangGraph, Node, Context

In [ ]:
# Set up OpenAI API key
openai.api_key = 'your-openai-api-key'

In [ ]:
# Initialize a LangGraph instance
graph = LangGraph()

In [ ]:
# Step 1: Define the Context (stores user preferences and generated responses)
context = Context()

In [ ]:
# Step 2: Define the LLM Node (GPT-3)
def query_with_llm(context):
    # Retrieve user input from the context
    user_input = context.get("user_input")
    
    # Use GPT-3 to generate a response based on the user input
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"User asked: {user_input}\nProvide a helpful and informative response.",
        max_tokens=100
    )
    
    # Get the generated text from the response
    generated_text = response.choices[0].text.strip()
    
    # Save the generated response in context
    context.set("llm_response", generated_text)
    
    return generated_text

llm_node = Node(
    name="Query with LLM",
    action=query_with_llm
)

In [ ]:
# Step 3: Define a Node to Display the LLM Response
display_llm_response_node = Node(
    name="Display LLM Response",
    action=lambda context: print("LLM Response: ", context.get("llm_response"))
)

In [ ]:
# Step 4: Define a Node for Asking User Input (Preference or Query)
ask_user_input_node = Node(
    name="Ask User for Input",
    action=lambda context: context.set("user_input", input("Ask me something: "))
)

In [ ]:
# Step 5: Set the flow of the graph (edges)
ask_user_input_node.add_edge(llm_node)
llm_node.add_edge(display_llm_response_node)

# Add the nodes to the graph
graph.add_node(ask_user_input_node)
graph.add_node(llm_node)
graph.add_node(display_llm_response_node)

In [ ]:
# Step 6: Run the graph
def run_graph():
    ask_user_input_node.execute(context)
    llm_node.execute(context)
    display_llm_response_node.execute(context)

# Run the graph to interact with the LLM
run_graph()